# Converting Rosanne's magnification files to magnified light curves and with the PLASTICC/SNANA format

### USER

In [1]:
DirLensFiles = "/Users/arturo/Dropbox/Research/Articulos/14_PLASTICC/MockData/\
Samples/2018_01_30/"

DirSaveOutput = DirData

# -------------------------------
# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand
NotebookName = 'Create_magnified_LCs.ipynb'

### Automatic

In [2]:
import numpy as np
import os # To use command line like instructions
import glob # To read the files in my directory
from matplotlib import pyplot as plt

5+6

11

#### Get the name of this ipython notebook
To print it in the output text files as reference.

In [3]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [4]:
print(NotebookName)

# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand

Create_magnified_LCs_v1_2.ipynb


In [5]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

In [6]:
# Function to convert from magnification to apparent magnitude

def Magnification2mag(Magf, mo):
    mag_int = mo - 2.5*np.log10(Magf)
    return mag_int

print '# Test:'
print "#", Magnification2mag(2.14118954, 19.46)

# Test
# 18.6333622172
# 18.633362217176515

# Test:
# 18.633362217176515


In [7]:
# Read the LSST star catalogue file with the magnitudes in ugrizY bands

starsdata = np.genfromtxt(DirData+'lsst_star_table_sample.txt',
                dtype=[float,float,float,float,float,
                       float,float,float]) 

print "# %s stars data found in the file"%len(starsdata)
# 35 stars data found in the file

# 35 stars data found in the file


In [8]:
starsdata['f2']

array([16.21739, 18.41886, 27.47009, 19.72312, 30.9791 , 26.53289,
       31.61692, 18.70061, 18.97976, 19.42187, 24.40802, 19.36606,
       17.46127, 29.19847, 30.27772, 23.00819, 24.09528, 21.57186,
       14.83046, 27.84922, 27.55117, 23.27528, 24.47566, 23.22603,
       27.1417 , 27.52953, 25.5453 , 27.31056, 26.48858, 24.22798,
       25.92916, 27.09815, 23.15348, 25.65296, 27.26104])

In [9]:
"""
array([ 16.21739,  18.41886,  27.47009,  19.72312,  30.9791 ,  26.53289,
        31.61692,  18.70061,  18.97976,  19.42187,  24.40802,  19.36606,
        17.46127,  29.19847,  30.27772,  23.00819,  24.09528,  21.57186,
        14.83046,  27.84922,  27.55117,  23.27528,  24.47566,  23.22603,
        27.1417 ,  27.52953,  25.5453 ,  27.31056,  26.48858,  24.22798,
        25.92916,  27.09815,  23.15348,  25.65296,  27.26104])
"""

'\narray([ 16.21739,  18.41886,  27.47009,  19.72312,  30.9791 ,  26.53289,\n        31.61692,  18.70061,  18.97976,  19.42187,  24.40802,  19.36606,\n        17.46127,  29.19847,  30.27772,  23.00819,  24.09528,  21.57186,\n        14.83046,  27.84922,  27.55117,  23.27528,  24.47566,  23.22603,\n        27.1417 ,  27.52953,  25.5453 ,  27.31056,  26.48858,  24.22798,\n        25.92916,  27.09815,  23.15348,  25.65296,  27.26104])\n'

In [10]:
import os # To use command line like instructions
import glob # To read the files in my directory

# Change the working directory where the data files are located
os.chdir(DirData+'1_Magnification/')

# Read the name of the fort (= magnitification) files
list_forts = glob.glob('fort*')

print "# %s fort files found."%len(list_forts)
# 4 fort files found.

#------------------------------

#- Force the creation of the directory to save the plots.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput+"2_plots"): os.makedirs(DirSaveOutput+"2_plots")

# 35 fort files found.


In [11]:
list_forts

['fort.21.txt',
 'fort.22.txt',
 'fort.23.txt',
 'fort.24.txt',
 'fort.25.txt',
 'fort.26.txt',
 'fort.27.txt',
 'fort.28.txt',
 'fort.29.txt',
 'fort.30.txt',
 'fort.31.txt',
 'fort.32.txt',
 'fort.33.txt',
 'fort.34.txt',
 'fort.35.txt',
 'fort.36.txt',
 'fort.37.txt',
 'fort.38.txt',
 'fort.39.txt',
 'fort.40.txt',
 'fort.41.txt',
 'fort.42.txt',
 'fort.43.txt',
 'fort.44.txt',
 'fort.45.txt',
 'fort.46.txt',
 'fort.47.txt',
 'fort.48.txt',
 'fort.49.txt',
 'fort.50.txt',
 'fort.51.txt',
 'fort.52.txt',
 'fort.53.txt',
 'fort.54.txt',
 'fort.55.txt']

In [12]:
# Reading the parameter values used to create every "fort" file
# these values are just to keep in the record and write down on 
# the Plasticc/SNANA output file.

ParsModel=np.genfromtxt(DirData+'1_Magnification/parameters_Binary_lens.txt')
print "# %s value sets were found in this file."%len(ParsModel)

# 35 value sets were found in this file.


### Main loop. v 1.2

In [13]:
# Difference in u magnitude as a criterium to write down 
# that row in the text file. This is helpful to avoid writting a lot of
# rows with exactly the same values of the magnitudes in ugrizY bands.
diffmag_fix = 0.001

# Maximum porcentage of time of a LC with magnitude larger
# than 16 mags
Porcentage_maxTime = 20.  

# Magnitude threshold
maxMag = 16. # mag

#------------------------------------------

textfile_1 = open(DirSaveOutput+'2_Lensing_lc_SNANA_format_.txt', 'w')

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y/%m/%d, %H:%M hrs.")
text_Date   = 'COMMENT: File created by Arturo Avelino on %s \n'%text_timenow
text_line = '#'+'-'*50 + '\n'
    
textfile_1.write("SURVEY: LSST \n")
textfile_1.write("FILTERS: ugrizY \n")
textfile_1.write("MODEL: Microlensing by Rosanne Di Stefano \n")
textfile_1.write("MODEL_PARNAMES: apeak,m1,m2,alpha,d_l,v,i_p,omega,clock,\
beta,nom,porb,tau \n")
textfile_1.write(text_Date)
# textfile_1.write()

countEvent = 0

print "# Number of useful rows written in the text file for given star,"
print "# it is, when there is a change in magnitude at least > %s mag."%diffmag_fix

for j in range(len(list_forts)): # Loop over the stars
# for j in range(4): # Loop over the stars # tmp  
    
    ra  = starsdata['f0'][j]
    dec = starsdata['f1'][j]
    
    # Apparent magnitude star in different bands:
    magStar_u = starsdata['f2'][j]
    magStar_g = starsdata['f3'][j]
    magStar_r = starsdata['f4'][j]
    magStar_i = starsdata['f5'][j]
    magStar_z = starsdata['f6'][j]
    magStar_y = starsdata['f7'][j]
    
    # Uploading the magnification table
    lensdata = np.genfromtxt(DirData+'1_Magnification/'+list_forts[j])
    
    ##########################################################
    
    # Loop over the magnification file for a given star
    # I use this loop just to count the number of rows and to
    # discard the magnified stars with magnitudes larger than
    # 16 mags for more than a given porcentage of time.
    
    # Reset the counter for the number of rows for this event
    # based on the change in magnitude in u band:
    countNROW = 0
    count_AboveMaxMag = 0
    
    for ii in range(len(lensdata)):

        Magnific = lensdata[ii][3]
        mag_u = Magnification2mag(Magnific, magStar_u)
        
        # Compute the difference between the previous magnitude 
        # and the current value:
        if ii > 0:
            Magnific_prev = lensdata[int(ii-1)][3]
            mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
            diffmag = abs(round(mag_u,3) - round(mag_u_prev,3))
        else: diffmag = 1
        
        if diffmag >= diffmag_fix: 
            countNROW += 1
            if mag_u < maxMag:
                count_AboveMaxMag += 1
                
    # Determine the porcentage of MJD with mag larger than 
    # the magnitude threshold:
    porcentage_int1 = (float(count_AboveMaxMag)*100.)/(float(countNROW+3))
    
    #########################################

    # Consider the LCs that have less than the maximum 
    # porcentage of time of a LC with magnitude larger
    # than the threshold magnitude only.
    
    if porcentage_int1 < Porcentage_maxTime:
        
        countEvent = countEvent + 1

        textfile_1.write(" \n")
        textfile_1.write(text_line)
        textfile_1.write(" \n")
        textfile_1.write("START_EVENT: %s \n"%countEvent)
        
        # Write the metadata info for this star.
        textfile_1.write("NROW: %s  RA: %s   DEC: %s  \n"%(
            (countNROW+3), ra, dec))
    
        #-----------------------------------------

        # Write the parameters of the specific event:

        textfile_1.write("PARVAL: ")
        for k in range(len(ParsModel[j])):
            textfile_1.write("%.3f  "%ParsModel[j][k])
        textfile_1.write(" \n")

        #---------------------------------

        time_first = int(lensdata[0][2]-1) # initial time

        # Magnitudes of the template
        textfile_1.write("T: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            time_first, magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))
        
        # Write the first event: it is equal in mags to template
        textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            time_first, magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))

        #---------------------------------------------------
        # Loop over the magnification file for a given star

        # Creating the plot 
        plt.figure()

        time_list = []
        mag_u_list = []
        mag_g_list = []

        time_list  += [time_first]
        mag_u_list += [magStar_u]
        mag_g_list += [magStar_g]

        countNROW_2 = 0
        for ii in range(len(lensdata)):

            time_int = lensdata[ii][2]
            Magnific = lensdata[ii][3]

            mag_u = Magnification2mag(Magnific, magStar_u)
            mag_g = Magnification2mag(Magnific, magStar_g)
            mag_r = Magnification2mag(Magnific, magStar_r)
            mag_i = Magnification2mag(Magnific, magStar_i)
            mag_z = Magnification2mag(Magnific, magStar_z)
            mag_y = Magnification2mag(Magnific, magStar_y)

            # Compute the difference between the previous magnitude 
            # and the current value, and print only the times
            # when the difference is larger than "diffmag_fix"
            if ii > 0:
                Magnific_prev = lensdata[int(ii-1)][3]
                mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
                diffmag = abs(round(mag_u,3) - round(mag_u_prev,3))
            else: diffmag = 1

            if diffmag >= diffmag_fix:

                text_05 = 'S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  %.3f \n'%(
                    time_int, mag_u, mag_g, mag_r, mag_i, mag_z, mag_y)
                textfile_1.write(text_05)

                countNROW_2 = countNROW_2 + 1

                time_list  += [time_int]
                mag_u_list += [mag_u] 
                mag_g_list += [mag_g]  
                
                # A hack to define the last MJD. I will use this to 
                # write down the last event where the MJD will the the
                # last used time + 1 day, and the magnitudes will
                # be the same than the template.
                time_int2 = time_int

        # Write the last event: it is equal in mags to template
        textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            (time_int2+1), magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))        
        
        
        #------------- PLOT THE LCs -------------------
        
        mag_u_np = np.array(mag_u_list)
        
        # the number is to create an offset:
        # mag_g_np = np.array(mag_g_list) + 2 

        # Plot with points and lines (recommended)
        plt.plot(time_list, mag_u_list, lw=0.5, marker=".", ms=2) 
        
        # just the line:
        # plt.plot(time_list, mag_u_list, lw=2, ls = "-", color = 'blue') 
        # plt.plot(time_list, mag_g_list, lw=2, ls = "-", color = 'green') 

        plt.xlim(min(time_list)-100, max(time_list)+50)
        plt.ylim(max(mag_u_list)+0.1, min(mag_u_list)-0.1)

        plt.xlabel('time (days)')
        plt.ylabel('apparent magnitude')
        plt.title('u-band light curves of a lensed LSST star')

        plt.savefig(DirSaveOutput+'2_plots/plot_%s.png'%countEvent, dpi=120)
        plt.close()
        textfile_1.write("END_EVENT: %s \n"%countEvent)
        print "%s: %s, %s countNROW"%(j, countNROW,countNROW_2)

textfile_1.close()

# Number of useful rows written in the text file for given star,
# it is, when there is a change in magnitude at least > 0.001 mag.
1: 922, 922 countNROW
2: 748, 748 countNROW
3: 1071, 1071 countNROW
4: 3889, 3889 countNROW
5: 672, 672 countNROW
6: 3879, 3879 countNROW
7: 731, 731 countNROW
8: 1633, 1633 countNROW
9: 1666, 1666 countNROW
10: 734, 734 countNROW
11: 11644, 11644 countNROW
13: 5034, 5034 countNROW
14: 1390, 1390 countNROW
15: 2095, 2095 countNROW
16: 1013, 1013 countNROW
17: 381, 381 countNROW
19: 1600, 1600 countNROW
20: 559, 559 countNROW
21: 880, 880 countNROW
22: 2294, 2294 countNROW
23: 444, 444 countNROW
24: 2078, 2078 countNROW
25: 6762, 6762 countNROW
26: 1387, 1387 countNROW
27: 2035, 2035 countNROW
28: 1610, 1610 countNROW
29: 1857, 1857 countNROW
30: 3541, 3541 countNROW
31: 1174, 1174 countNROW
32: 2274, 2274 countNROW
33: 1048, 1048 countNROW
34: 231, 231 countNROW
